# 因子分解机

## 线性模型

$$\hat{y}(\pmb{x}) = w_0 + \sum_{i=1}^n w_i x_i$$

**缺点**

各个特征分量 $x_i$ 和 $x_j \ (i \neq j)$ 彼此之间是独立的，忽略了特征之间的相互组合关系

## 多项式模型

$$\hat{y}(\pmb{x}) = w_0 + \sum_{i=1}^n w_i x_i + \sum_{i=1}^n \sum_{j=i+1}^n w_{ij} x_i x_j$$

其中 $x_i x_j$ 表示两个互异特征组合的二阶特征，$w_{ij}$ 表示二阶特征的交叉项系数

**缺点**

- 数据稀疏性普遍存在的实际应用场景中，而二阶特征系数 $w_{ij}$ 的训练又需要大量特征分量 $x_i$ 和 $x_j$ 都非零的样本，因此二阶特征系数 $w_{ij}$ 很难收敛
- 权重参数的数量由 $n$ 直接上升到 $n^2$，极大地增加了训练复杂度

## FM 模型
为了克服模型无法在稀疏数据场景下学习二阶特征系数 $w_{ij}$，我们需要将 $w_{ij}$ 表示为另外一种形式。为此，针对样本 $\pmb{x}$ 的第 $i$ 维特征分量 $x_i$，引入辅助隐向量 $\pmb{v}_i$

$$\pmb{v}_i = (v_{i1},v_{i2},\cdots,v_{ik})$$

其中 $k$ 为超参数，表示特征分量 $x_i$ 对应一个 $k$ 维隐向量 $\pmb{v}_i$

使用两个特征隐向量的内积作为交叉特征的权重

$$w_{ij} = \langle \pmb{v}_i,\pmb{v}_j \rangle$$

将二阶多项式模型改写为 FM 模型

$$\hat{y}(\pmb{x}) = w_0 + \sum_{i=1}^n w_i x_i + \sum_{i=1}^n \sum_{j=i+1}^n \langle \pmb{v}_i,\pmb{v}_j \rangle x_i x_j$$

**有效性分析**

假设在训练数据里特征 $x_i$ 和 $x_j$ 并未同时出现，如果按照多项式模型的训练方式，是无法学习到该特征组合的权重的。但是因为 FM 模型学习的是单个特征的隐向量，并不依赖某个特定的特征组合，所以只要特征 $x_i$ 和其它任意特征的组合出现过，那么就可以学到自己对应的隐向量。于是，尽管 $x_i x_j$ 这组特征组合没有在训练时出现，但是在预测的时候，如果出现这组特征组合，FM 模型仍然可以通过内积算出这组新的特征组合的权重。这就是 FM 模型泛化能力强的根本原因，也是 FM 在稀疏特征应用环境下有效的原因

**优化效率**

从上述 FM 模型公式可以看出模型的复杂度是 $O(n^2 k)$，无法满足实际应用需求，于是我们需要进一步优化公式，以降低复杂度

$$
\begin{aligned}
\sum_{i=1}^n \sum_{j=i+1}^n \langle \pmb{v}_i,\pmb{v}_j \rangle x_i x_j
=& \frac{1}{2} \sum_{i=1}^n \sum_{j=1}^n \langle \pmb{v}_i,\pmb{v}_j \rangle x_i x_j - \frac{1}{2} \sum_{i=1}^n \sum_{j=1}^n \langle \pmb{v}_i,\pmb{v}_j \rangle x_i x_i \\
=& \frac{1}{2} (\sum_{i=1}^n \sum_{j=1}^n \sum_{f=1}^k v_{if} v_{jf} x_i x_j - \sum_{i=1}^n \sum_{f=1}^k v_{if} v_{if} x_i x_i) \\
=& \frac{1}{2} \sum_{f=1}^k [(\sum_{i=1}^n v_{if} x_i) (\sum_{j=1}^n v_{jf} x_j) - \sum_{i=1}^n v_{if}^2 x_i^2] \\
=& \frac{1}{2} \sum_{f=1}^k [(\sum_{i=1}^n v_{if} x_i)^2 - \sum_{i=1}^n v_{if}^2 x_i^2]
\end{aligned}
$$

$$\hat{y}(\pmb{x}) = w_0 + \sum_{i=1}^n w_i x_i + \frac{1}{2} \sum_{f=1}^k [(\sum_{i=1}^n v_{if} x_i)^2 - \sum_{i=1}^n v_{if}^2 x_i^2]$$

优化之后模型是的时间复杂度为 $O(nk)$


**求导**

$$
\frac{\partial \hat{y}(\pmb{x})}{\partial \theta} =
\begin{cases}
1 & \mathrm{if} \ \theta \ \mathrm{is} \ w_0 \\
x_i & \mathrm{if} \ \theta \ \mathrm{is} \ w_i \\
x_i \sum_{j=1}^n v_{jf} x_j - v_{if} x_i^2 & \mathrm{if} \ \theta \ \mathrm{is} \ v_{if}
\end{cases}
$$

**如何做召回？**

1. 离线训练好 FM 模型（学习目标可以是 CTR 预估）
2. 将用户侧和物品侧对应的 Embedding（隐向量）分别做平均池化，得到最终的 Embedding
3. 将所有的 Embedding 向量放入 Faiss 等
4. 用户在线上发出请求，取出用户对应的 Embedding，进行检索召回

In [ ]:
# 定义FM模型
class FM_model(torch.nn.Module):
    """FM Model"""
    def __init__(self, n, k):
        super(FM_model, self).__init__()
        self.n = n
        self.k = k
        self.linear = torch.nn.Linear(self.n, 1, bias=True)
        self.v = torch.nn.Parameter(torch.rand(self.n, self.k))

    def fm_layer(self, x):
        # w_i * x_i 线性部分
        linear_part = self.linear(x)
        print(linear_part.shape)
        # pairwise interactions part 1
        inter_part1 = torch.mm(x, self.v)
        # pairwise interactions part 2
        inter_part2 = torch.mm(torch.pow(x, 2), torch.pow(self.v, 2))
        inter_part = 0.5 * torch.sum(torch.sub(torch.pow(inter_part1, 2), inter_part2), dim=1).reshape(-1, 1)
        output = linear_part + inter_part
        return output

    def forward(self, x):
        output = self.fm_layer(x)
        return output

## MF 模型
本质上，MF 模型（矩阵分解模型）是 FM 模型的特例，MF 模型可以被认为是只有 user_id 和 item_id 这两个特征的 FM 模型，MF 模型将这两类特征通过矩阵分解，来达到将这两类特征向量化表示的目的。而 FM 模型则可以看作是 MF 模型的进一步拓展，除了 user_id 和 item_id 这两类特征外，很多其它类型的特征，都可以进一步融入 FM 模型里，它将所有这些特征转化为低维向量表达，并计算任意两个特征向量的内积，即特征组合的权重

## FFM 模型
FFM 模型是 FM 模型的升级版，引入了 field 的概念。FFM 模型把相同性质的特征归于同一个 field。在 FFM 模型中，每一维特征，针对每一种 field $f_j$，都会学习到一个隐向量 $\pmb{v}_{i,f_j}$，因此，隐向量不仅与特征相关，也与 field 相关。也就是说，“Day=26/11/15”这个特征与“Country”特征和“Ad_type”特征进行关联的时候使用不同的隐向量，这与“Country”和“Ad_type”的内在差异相符，也是 FFM 模型中“field-aware”的由来

假设样本的 $n$ 个特征属于 $f$ 个 field，那么 FFM 模型的二次项有 $nf$ 个隐向量。而在 FM 模型中，每一维特征的隐向量只有一个。FM 模型可以看作 FFM 模型的特例，是把所有特征都归属到一个 field 时的 FFM 模型。根据 FFM 模型的 field 敏感特性，可以导出其模型方程

$$\hat{y}(\pmb{x}) = w_0 + \sum_{i=1}^n w_i x_i + \sum_{i=1}^n \sum_{j=i+1}^n \langle \pmb{v}_{i,f_j} \ ,\pmb{v}_{j,f_i} \rangle x_i x_j$$

其中，$f_j$ 是第 $j$ 个特征所属的 field，$\pmb{v}_{i,f_j}$ 表示与域 $f_j$ 对应的第 $i$ 个特征的隐向量。如果隐向量的长度为 $k$，那么 FFM 模型的二次参数有 $nfk$ 个，远多于 FM 模型的 $nk$ 个。此外，由于隐向量与 field 相关，FFM 二次项并不能够化简，其预测复杂度是 $O(n^2 k)$

**案例**

假设一输入记录如下

<img src="../../img/ffm1.png" width="300" >

这条记录可以编码成 5 个特征，其中“Genre=Comedy”和“Genre=Drama”属于同一个 field，“Price”是数值型，不用 one-hot 编码转换。为了方便说明 FFM 模型的样本格式，我们将所有的特征和对应的 field 映射成整数编号

<img src="../../img/ffm2.png" width="500" >

那么，FFM 模型的组合特征有 10 项，如下图所示

<img src="../../img/ffm3.png" >

其中，红色是 field 编号，蓝色是特征编号，绿色是此样本的特征取值。二次项的系数是通过与特征 field 相关的隐向量点积得到的，二次项共有 $\frac{n(n-1)}{2}$ 个

相比 FM，FFM 引入了特征域的概念，为模型引入了更多有价值的信息，使模型的表达能力更强，但与此同时，FFM 的计算复杂度上升到 $n^2 k$，远大于 FM 的 $nk$。在实际工程应用中，需要在模型效果和工程投入之间进行权衡